In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [23]:
def import_and_transform_air_quality_data():
    streets = ['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse']
    street_dataframes = []

    for street in streets:
        yearly_data = []

        for year in range(2012, 2023):
            file_path = f'airquality/daily/ugz_ogd_air_d1_{year}.csv'
            data = pd.read_csv(file_path)

            # Filter for a specific street
            street_data = data[data['Standort'] == street]

            # Pivot the data
            street_wide = street_data.pivot(index='Datum', columns='Parameter', values='Wert')

            # Add the year and street columns
            street_wide['Year'] = year
            street_wide['Street'] = street

            yearly_data.append(street_wide)

        # Concatenate all years' data for the street
        combined_street_data = pd.concat(yearly_data)
        street_dataframes.append(combined_street_data)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes)

    return combined_data

In [24]:
airquality = import_and_transform_air_quality_data()
#airquality.to_csv('airquality/airquality.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'airquality/daily/ugz_ogd_air_d1_2012.csv'

In [25]:
def import_and_transform_traffic_data():
    streets = ['Stampfenbachstrasse (Wasserwerkstrasse)', 'Schimmelstrasse (Werdstrasse)', 'Rosengartenstrasse']
    street_dataframes = []

    for street in streets:
        yearly_data = []

        for year in range(2012, 2023):
            file_path = f'traffic/sid_dav_verkehrszaehlung_miv_od2031_{year}.csv'
            traffic = pd.read_csv(file_path)

            # Drop unnecessary columns and filter for a specific street
            traffic_small = traffic.drop(columns=['MSID', 'MSName', 'Hoehe', 'EKoord', 'NKoord', 'AnzDetektoren', 'D1ID', 'D2ID', 'D3ID','D4ID', 'Knummer', 'ZSID'])
            traffic_rg = traffic_small[traffic_small['ZSName'] == street]
            traffic_rg['MessungDatZeit'] = pd.to_datetime(traffic_rg['MessungDatZeit'])

            # Aggregate data
            daily_traffic = traffic_rg.groupby(traffic_rg['MessungDatZeit'].dt.date)['AnzFahrzeuge'].sum().reset_index(name='TotalDailyTraffic')
            daily_traffic['Year'] = year
            daily_traffic['Street'] = street

            yearly_data.append(daily_traffic)

        # Concatenate all years' data for the street
        combined_street_data = pd.concat(yearly_data)
        street_dataframes.append(combined_street_data)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes)

    return combined_data


In [ ]:
traffic = import_and_transform_traffic_data()
#traffic.to_csv('traffic/traffic_data.csv')

In [ ]:
airquality = pd.read_csv('airquality/airquality.csv')
airquality['Datum'] = pd.to_datetime(airquality['Datum']).dt.date
#airquality.to_csv('airquality/airquality_dt.csv', index=False)


In [ ]:
traffic = pd.read_csv('traffic/traffic_data.csv') 
traffic.replace(['Stampfenbachstrasse (Wasserwerkstrasse)','Schimmelstrasse (Werdstrasse)', 'Rosengartenstrasse'],['Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse', 'Zch_Rosengartenstrasse'], inplace=True)
traffic.rename(columns={"MessungDatZeit": "Datum"}, inplace=True)
#traffic.to_csv('traffic/traffic_rtm.csv', index=False)


In [ ]:
airquality_dt = pd.read_csv('airquality/airquality_dt.csv')

In [12]:
merged_df = pd.merge(traffic_rtm, airquality_dt, how="left", on=['Datum', 'Street'])
merged_df.drop(columns=merged_df[['Year_x', 'Year_y']])
#merged_df.to_csv('finaldata/merged_df.csv')

NameError: name 'traffic_rtm' is not defined

In [13]:
merged_df = pd.read_csv('finaldata/merged_df.csv')
merged_df.drop(columns=merged_df[['PM2.5', 'CO', 'SO2', 'PN']], inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'finaldata/merged_df.csv'

In [14]:
# Including 'Street' in the subset for plotting
merged_df['Datum'] = pd.to_datetime(merged_df['Datum'])
time_series_data_with_street = merged_df[['Datum', 'Street', 'TotalDailyTraffic', 'NO2', 'PM10']]

# Function to plot time series for each unique street
def plot_time_series_for_streets(data, column, title, color):
    streets = data['Street'].unique()
    n_streets = len(streets)

    # Creating subplots for each street
    fig, axs = plt.subplots(n_streets, 1, figsize=(15, 4 * n_streets))

    for i, street in enumerate(streets):
        street_data = data[data['Street'] == street]
        axs[i].plot(street_data['Datum'], street_data[column], label=street, color=color)
        axs[i].set_title(f'{title} for {street}')
        axs[i].set_xlabel('Date')
        axs[i].set_ylabel(column)
        axs[i].legend()
        axs[i].xaxis.set_major_locator(mdates.YearLocator())
        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # Adjusting layout
    plt.tight_layout()

    # Showing the plots
    plt.show()

# Plotting time series for 
plot_time_series_for_streets(time_series_data_with_street, 'TotalDailyTraffic', 'Total Daily Traffic', 'blue')
plot_time_series_for_streets(time_series_data_with_street, 'NO2', 'NO2 Levels', 'red')
plot_time_series_for_streets(time_series_data_with_street, 'PM10', 'PM10 Levels', 'green')

NameError: name 'merged_df' is not defined

In [15]:
#Adding meteo data to merged df

def import_and_transform_meteo_data():
    streets = ['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse']
    street_dataframes = []

    for street in streets:
        yearly_data = []

        for year in range(2012, 2023):
            file_path = f'meteo/ugz_ogd_meteo_d1_{year}.csv'
            data = pd.read_csv(file_path)

            # Filter for a specific street
            street_data = data[data['Standort'] == street]

            # Pivot the data
            street_wide = street_data.pivot(index='Datum', columns='Parameter', values='Wert')

            # Add the year and street columns
            street_wide['Year'] = year
            street_wide['Street'] = street

            yearly_data.append(street_wide)

        # Concatenate all years' data for the street
        combined_street_data = pd.concat(yearly_data)
        street_dataframes.append(combined_street_data)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes)

    return combined_data

meteo = import_and_transform_meteo_data()
#meteo.to_csv('meteo/meteo.csv')

meteo = pd.read_csv('meteo/meteo.csv')
meteo['Datum'] = pd.to_datetime(meteo['Datum']).dt.date
#meteo.to_csv('meteo/meteo_dt.csv', index=False)

meteo_dt = pd.read_csv('finaldata/meteo_dt.csv')

merged_meteo = pd.merge(merged_df, meteo_dt, how="left", on=['Datum', 'Street'])
#merged_meteo.to_csv('finaldata/merged_meteo.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'meteo/ugz_ogd_meteo_d1_2012.csv'

In [16]:
merged_meteo = pd.read_csv('/Users/fredericksafian/VSCODE/finaldata/merged_meteo.csv')
merged_meteo.drop(columns=merged_meteo[['Unnamed: 0', 'Year']], inplace=True)
merged_meteo['Datum'] = pd.to_datetime(merged_meteo['Datum']).dt.date

# Adding season dummies
def get_month(date):
    month_dict = {
        1: 'Jan',
        2: 'Feb',
        3: 'Mar',
        4: 'Apr',
        5: 'May',
        6: 'Jun',
        7: 'Jul',
        8: 'Aug',
        9: 'Sep',
        10: 'Oct',
        11: 'Nov',
        12: 'Dec'
    }
    return month_dict[date.month]

merged_meteo['Month'] = merged_meteo['Datum'].apply(get_month)
month_dummies = pd.get_dummies(merged_meteo['Month'])
merged_meteo = pd.concat([merged_meteo, month_dummies], axis=1).drop(columns=['Month'])

#adding weekday dummies 

merged_meteo['Weekday'] = pd.to_datetime(merged_meteo['Datum']).dt.dayofweek
weekday_dummies = pd.get_dummies(merged_meteo['Weekday'], prefix='Weekday')
merged_meteo = pd.concat([merged_meteo, weekday_dummies], axis=1)
merged_meteo.drop(['Weekday'], axis=1, inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fredericksafian/VSCODE/finaldata/merged_meteo.csv'

In [17]:
#adding tree and green spaces

#calculating trees around measurement points
import re

tree_locations_path = '/Users/fredericksafian/VSCODE/streetdata/gsz.baumkataster_baumstandorte.csv'
tree_locations_df = pd.read_csv(tree_locations_path)

def extract_coordinates(point_str):
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", point_str)
    return tuple(map(float, numbers))

tree_locations_df['coordinates'] = tree_locations_df['geometry'].apply(extract_coordinates)

def calculate_distance(coord1, coord2):
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5

target_coords = {
    "Zch_Rosengartenstrasse": (2682095, 1249940),
    "Zch_Schimmelstrasse": (2681950, 1247250),
    "Zch_Stampfenbachstrasse": (2683140, 1249040)
}

tree_counts_df = pd.DataFrame(columns=['Street', '1.5km', '1km', '500m', '200m'])

for street, coords in target_coords.items():
    tree_counts = {
        'Street': street,
        '1.5km': sum(calculate_distance(tree_coord, coords) <= 1500 for tree_coord in tree_locations_df['coordinates']),
        '1km': sum(calculate_distance(tree_coord, coords) <= 1000 for tree_coord in tree_locations_df['coordinates']),
        '500m': sum(calculate_distance(tree_coord, coords) <= 500 for tree_coord in tree_locations_df['coordinates']),
        '200m': sum(calculate_distance(tree_coord, coords) <= 200 for tree_coord in tree_locations_df['coordinates'])
    }
    tree_counts_df = pd.concat([tree_counts_df, pd.DataFrame([tree_counts])], ignore_index=True)




# calculating tree spaces around measurement points

green_spaces_path = '/Users/fredericksafian/VSCODE/streetdata/gsz.gruenflaechen.csv'
green_spaces_df = pd.read_csv(green_spaces_path)

green_spaces_df['coordinates'] = green_spaces_df['geometry'].apply(extract_coordinates)

green_space_counts_df = pd.DataFrame(columns=['Street', '1.5km', '1km', '500m', '200m'])

for street, coords in target_coords.items():
    green_space_counts = {
        'Street': street,
        '1.5km': sum(calculate_distance(green_coord, coords) <= 1500 for green_coord in green_spaces_df['coordinates']),
        '1km': sum(calculate_distance(green_coord, coords) <= 1000 for green_coord in green_spaces_df['coordinates']),
        '500m': sum(calculate_distance(green_coord, coords) <= 500 for green_coord in green_spaces_df['coordinates']),
        '200m': sum(calculate_distance(green_coord, coords) <= 200 for green_coord in green_spaces_df['coordinates'])
    }
    green_space_counts_df = pd.concat([green_space_counts_df, pd.DataFrame([green_space_counts])], ignore_index=True)



#merging it with the data set

merged_meteo = pd.merge(merged_meteo, green_space_counts_df, on='Street', how='left', suffixes=('_green', ''))
merged_meteo = pd.merge(merged_meteo, tree_counts_df, on='Street', how='left', suffixes=('','_trees'))
distance_columns_to_rename = {
    '1.5km': '1.5km_green',
    '1km': '1km_green',
    '500m': '500m_green',
    '200m': '200m_green'
}
merged_meteo = merged_meteo.rename(columns=distance_columns_to_rename)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fredericksafian/VSCODE/streetdata/gsz.baumkataster_baumstandorte.csv'

In [18]:
#windspeed and humidity data is only in the hourly meteo set, so we will add that

def import_and_transform_meteo_hourly_data():
    streets = ['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse']
    street_dataframes = []
    parameters = ['WD', 'WVv', 'WVs', 'Hr']

    for year in range(2012, 2023): 
        for street in streets:
            file_path = f'/Users/fredericksafian/VSCODE/meteohourly/ugz_ogd_meteo_h1_{year}.csv'  
            data = pd.read_csv(file_path)

            # Filter for a specific street
            street_data = data[data['Standort'] == street]

            # Pivot the data to wide format
            street_wide = street_data.pivot(index='Datum', columns='Parameter', values='Wert')

            # Reset index to make 'Datum' a column again
            street_wide.reset_index(inplace=True)

            # Convert 'Datum' to datetime and keep only the date part
            street_wide['Datum'] = pd.to_datetime(street_wide['Datum']).dt.date

            # Select only the columns that exist in the dataset
            cols_to_keep = ['Datum'] + [col for col in parameters if col in street_wide.columns]
            street_wide = street_wide[cols_to_keep]

            # Group by Datum, calculate daily mean
            daily_mean = street_wide.groupby('Datum', as_index=False).mean()

            # Add the street name to the DataFrame
            daily_mean['Street'] = street

            street_dataframes.append(daily_mean)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes, ignore_index=True)

    return combined_data

# Use this function and export the result if needed
meteo_hourly = import_and_transform_meteo_hourly_data()



#merge it

merged_meteo = pd.merge(merged_meteo, meteo_hourly, how="left", on=['Datum', 'Street'])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fredericksafian/VSCODE/meteohourly/ugz_ogd_meteo_h1_2012.csv'

In [19]:
#checking the speed limits 

speed = pd.read_csv('/Users/fredericksafian/VSCODE/streetdata/taz.view_geoserver_tempo_ist_e.csv')
speed.rename(columns={'lokalisationsname': 'Street'}, inplace=True)
streets = ['Rosengartenstrasse', 'Schimmelstrasse', 'Stampfenbachstrasse']

speed = speed[speed['Street'].isin(streets)]
speed

#all the same anyway

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fredericksafian/VSCODE/streetdata/taz.view_geoserver_tempo_ist_e.csv'

In [20]:
# Create the NO2_tomorrow (target variable) column by shifting the NO2 column up by one row
unique_streets = merged_meteo['Street'].unique()
data_separated = []
for street in unique_streets:
    df_street = merged_meteo[merged_meteo['Street'] == street].copy()
    df_street['NO2_tomorrow'] = df_street['NO2'].shift(-1)
    data_separated.append(df_street)

merged_meteo = pd.concat(data_separated)

merged_meteo.head()



#merged_meteo.to_csv('/Users/fredericksafian/VSCODE/finaldata/merged_with_dummies.csv')


NameError: name 'merged_meteo' is not defined

In [21]:
merged_with_dummies = pd.read_csv('/Users/fredericksafian/VSCODE/finaldata/merged_with_dummies.csv')
merged_with_dummies.columns.unique()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fredericksafian/VSCODE/finaldata/merged_with_dummies.csv'